# Preperation

In [90]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import scipy.spatial
import warnings
import sklearn as sk

warnings.simplefilter("ignore")
# Enable inline mode for matplotlib so that Jupyter displays graphs
%matplotlib inline

pd.__version__ #print which version of pandas you're using

'0.24.2'

## read and combine files

In [127]:
h1b_17 = pd.read_csv('H-1B_2017.csv')
h1b_17.shape

(624650, 53)

In [128]:
h1b_17 = h1b_17.drop('Unnamed: 0',axis=1)
h1b_17.head()

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_BUSINESS_DBA,EMPLOYER_ADDRESS,...,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,PUBLIC_DISCLOSURE_LOCATION,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,ORIGINAL_CERT_DATE
0,I-200-16055-173457,CERTIFIED-WITHDRAWN,2016-02-24,2016-10-01,H-1B,2016-08-10,2019-08-10,DISCOVER PRODUCTS INC.,NaN,2500 LAKE COOK ROAD,...,N,N,NaN,Y,NaN,RIVERWOODS,LAKE,IL,60015,2016-03-01
1,I-200-16064-557834,CERTIFIED-WITHDRAWN,2016-03-04,2016-10-01,H-1B,2016-08-16,2019-08-16,DFS SERVICES LLC,NaN,2500 LAKE COOK ROAD,...,N,N,NaN,Y,NaN,RIVERWOODS,LAKE,IL,60015,2016-03-08
2,I-200-16063-996093,CERTIFIED-WITHDRAWN,2016-03-10,2016-10-01,H-1B,2016-09-09,2019-09-09,EASTBANC TECHNOLOGIES LLC,NaN,1211 31ST ST. NW,...,Y,N,Y,NaN,NaN,WASHINGTON,NaN,DC,20007,2016-03-16
3,I-200-16272-196340,WITHDRAWN,2016-09-28,2016-10-01,H-1B,2017-01-26,2020-01-25,INFO SERVICES LLC,NaN,17177 NORTH LAUREL PARK DR,...,Y,N,Y,NaN,NaN,JERSEY CITY,HUDSON,NJ,07302,NaN
4,I-200-15053-636744,CERTIFIED-WITHDRAWN,2015-02-22,2016-10-02,H-1B,2015-03-01,2018-03-01,BB&T CORPORATION,NaN,223 WEST NASH STREET,...,N,N,NaN,Y,NaN,NEW YORK,NEW YORK,NY,10036,2015-02-26


In [93]:
h1b_18 = pd.read_csv('H-1B_2018.csv')
h1b_18.shape

(654360, 52)

In [94]:
h1b_18.rename(columns={'NEW_CONCURRENT_EMP': 'NEW_CONCURRENT_EMPLOYMENT'}, inplace=True)
h1b_18.head()

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_BUSINESS_DBA,EMPLOYER_ADDRESS,...,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,PUBLIC_DISCLOSURE_LOCATION,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,ORIGINAL_CERT_DATE
0,I-200-18026-338377,CERTIFIED,1/29/18,2/2/18,H-1B,7/28/18,7/27/21,MICROSOFT CORPORATION,NaN,1 MICROSOFT WAY,...,N,N,NaN,NaN,NaN,REDMOND,KING,WA,98052,NaN
1,I-200-17296-353451,CERTIFIED,10/23/17,10/27/17,H-1B,11/6/17,11/6/20,ERNST & YOUNG U.S. LLP,NaN,200 PLAZA DRIVE,...,N,N,NaN,NaN,NaN,SANTA CLARA,SAN JOSE,CA,95110,NaN
2,I-200-18242-524477,CERTIFIED,8/30/18,9/6/18,H-1B,9/10/18,9/9/21,LOGIXHUB LLC,NaN,320 DECKER DRIVE,...,N,N,NaN,NaN,NaN,IRVING,DALLAS,TX,75062,NaN
3,I-200-18070-575236,CERTIFIED,NaN,3/30/18,H-1B,9/10/18,9/9/21,"HEXAWARE TECHNOLOGIES, INC.",NaN,101 WOOD AVENUE SOUTH,...,Y,N,Y,NaN,NaN,NEW CASTLE,NEW CASTLE,DE,19720,NaN
4,I-200-18243-850522,CERTIFIED,8/31/18,9/7/18,H-1B,9/7/18,9/6/21,"ECLOUD LABS,INC.",NaN,120 S WOOD AVENUE,...,Y,N,Y,Y,NaN,BIRMINGHAM,SHELBY,AL,35244,NaN


### deal with type issue on h1b_18

In [95]:
import decimal

def str_to_float(field, raw_data):
    wages = []
    for i,  r in h1b_18.iterrows():
        if type(r[field]) != float:
            wages.append(float(decimal.Decimal(r[field].replace(",",""))))
        else:
            wages.append(r[field])

    raw_data[field] = wages
    return raw_data

In [96]:
h1b_18 = str_to_float('PREVAILING_WAGE', h1b_18)
h1b_18 = str_to_float('WAGE_RATE_OF_PAY_FROM', h1b_18)
h1b_18 = str_to_float('WAGE_RATE_OF_PAY_TO', h1b_18)

### dealing with dates

In [126]:
def date_form_17(col_name, df, month=False):
    col = df[col_name]
    result = []
    for i in range(len(col)):
        if type(col[i]) == float:
            col[i] = col[0]
        date = col[i].split('-')
        need = date[1] if month == True else date[0][2:]
        result.append(need)
    return result

def date_form_18(col_name, df, month=False):
    col = df[col_name]
    result = []
    for i in range(len(col)):
        if type(col[i]) == float:
            col[i] = col[i+1]
        date = col[i].split('/')
        need = date[0] if month == True else date[2]
        result.append(need)
    return result


In [129]:
h1b_17['CASE_SUBMITTED'] = date_form_17('CASE_SUBMITTED', h1b_17, True)
print('finish 1')
h1b_17['EMPLOYMENT_START_DATE'] = date_form_17('EMPLOYMENT_START_DATE', h1b_17)
print('finish 2')
h1b_17['EMPLOYMENT_END_DATE'] = date_form_17('EMPLOYMENT_END_DATE', h1b_17)
h1b_17.head()

finish 1
finish 2


,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_BUSINESS_DBA,EMPLOYER_ADDRESS,...,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,PUBLIC_DISCLOSURE_LOCATION,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,ORIGINAL_CERT_DATE
0,I-200-16055-173457,CERTIFIED-WITHDRAWN,02,2016-10-01,H-1B,16,19,DISCOVER PRODUCTS INC.,NaN,2500 LAKE COOK ROAD,...,N,N,NaN,Y,NaN,RIVERWOODS,LAKE,IL,60015,2016-03-01
1,I-200-16064-557834,CERTIFIED-WITHDRAWN,03,2016-10-01,H-1B,16,19,DFS SERVICES LLC,NaN,2500 LAKE COOK ROAD,...,N,N,NaN,Y,NaN,RIVERWOODS,LAKE,IL,60015,2016-03-08
2,I-200-16063-996093,CERTIFIED-WITHDRAWN,03,2016-10-01,H-1B,16,19,EASTBANC TECHNOLOGIES LLC,NaN,1211 31ST ST. NW,...,Y,N,Y,NaN,NaN,WASHINGTON,NaN,DC,20007,2016-03-16
3,I-200-16272-196340,WITHDRAWN,09,2016-10-01,H-1B,17,20,INFO SERVICES LLC,NaN,17177 NORTH LAUREL PARK DR,...,Y,N,Y,NaN,NaN,JERSEY CITY,HUDSON,NJ,07302,NaN
4,I-200-15053-636744,CERTIFIED-WITHDRAWN,02,2016-10-02,H-1B,15,18,BB&T CORPORATION,NaN,223 WEST NASH STREET,...,N,N,NaN,Y,NaN,NEW YORK,NEW YORK,NY,10036,2015-02-26


In [130]:
h1b_18['CASE_SUBMITTED'] = date_form_18('CASE_SUBMITTED', h1b_18, True)
print('finish 1')
h1b_18['EMPLOYMENT_START_DATE'] = date_form_18('EMPLOYMENT_START_DATE', h1b_18)
print('finish 2')
h1b_18['EMPLOYMENT_END_DATE'] = date_form_18('EMPLOYMENT_END_DATE', h1b_18)
h1b_18.head()

finish 1
finish 2


,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_BUSINESS_DBA,EMPLOYER_ADDRESS,...,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,PUBLIC_DISCLOSURE_LOCATION,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,ORIGINAL_CERT_DATE
0,I-200-18026-338377,CERTIFIED,1,2/2/18,H-1B,18,21,MICROSOFT CORPORATION,NaN,1 MICROSOFT WAY,...,N,N,NaN,NaN,NaN,REDMOND,KING,WA,98052,NaN
1,I-200-17296-353451,CERTIFIED,10,10/27/17,H-1B,17,20,ERNST & YOUNG U.S. LLP,NaN,200 PLAZA DRIVE,...,N,N,NaN,NaN,NaN,SANTA CLARA,SAN JOSE,CA,95110,NaN
2,I-200-18242-524477,CERTIFIED,8,9/6/18,H-1B,18,21,LOGIXHUB LLC,NaN,320 DECKER DRIVE,...,N,N,NaN,NaN,NaN,IRVING,DALLAS,TX,75062,NaN
3,I-200-18070-575236,CERTIFIED,8,3/30/18,H-1B,18,21,"HEXAWARE TECHNOLOGIES, INC.",NaN,101 WOOD AVENUE SOUTH,...,Y,N,Y,NaN,NaN,NEW CASTLE,NEW CASTLE,DE,19720,NaN
4,I-200-18243-850522,CERTIFIED,8,9/7/18,H-1B,18,21,"ECLOUD LABS,INC.",NaN,120 S WOOD AVENUE,...,Y,N,Y,Y,NaN,BIRMINGHAM,SHELBY,AL,35244,NaN


### combine files together

In [132]:
frames = [h1b_17, h1b_18]
raw_data = pd.concat(frames, axis=0, sort=False)
print(raw_data.shape)
raw_data = raw_data.drop('DECISION_DATE', axis=1)
raw_data = raw_data.drop('ORIGINAL_CERT_DATE', axis=1)
print(raw_data.columns)
raw_data.head()

(1279010, 52)
Index(['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'VISA_CLASS',
       'EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE', 'EMPLOYER_NAME',
       'EMPLOYER_BUSINESS_DBA', 'EMPLOYER_ADDRESS', 'EMPLOYER_CITY',
       'EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE', 'EMPLOYER_COUNTRY',
       'EMPLOYER_PROVINCE', 'EMPLOYER_PHONE', 'EMPLOYER_PHONE_EXT',
       'AGENT_REPRESENTING_EMPLOYER', 'AGENT_ATTORNEY_NAME',
       'AGENT_ATTORNEY_CITY', 'AGENT_ATTORNEY_STATE', 'JOB_TITLE', 'SOC_CODE',
       'SOC_NAME', 'NAICS_CODE', 'TOTAL_WORKERS', 'NEW_EMPLOYMENT',
       'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT',
       'NEW_CONCURRENT_EMPLOYMENT', 'CHANGE_EMPLOYER', 'AMENDED_PETITION',
       'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY',
       'PW_WAGE_LEVEL', 'PW_SOURCE', 'PW_SOURCE_YEAR', 'PW_SOURCE_OTHER',
       'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY',
       'H1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'LABOR_CON_AGREE',
  

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_BUSINESS_DBA,EMPLOYER_ADDRESS,EMPLOYER_CITY,...,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,PUBLIC_DISCLOSURE_LOCATION,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE
0,I-200-16055-173457,CERTIFIED-WITHDRAWN,02,H-1B,16,19,DISCOVER PRODUCTS INC.,NaN,2500 LAKE COOK ROAD,RIVERWOODS,...,Year,N,N,NaN,Y,NaN,RIVERWOODS,LAKE,IL,60015
1,I-200-16064-557834,CERTIFIED-WITHDRAWN,03,H-1B,16,19,DFS SERVICES LLC,NaN,2500 LAKE COOK ROAD,RIVERWOODS,...,Year,N,N,NaN,Y,NaN,RIVERWOODS,LAKE,IL,60015
2,I-200-16063-996093,CERTIFIED-WITHDRAWN,03,H-1B,16,19,EASTBANC TECHNOLOGIES LLC,NaN,1211 31ST ST. NW,WASHINGTON,...,Year,Y,N,Y,NaN,NaN,WASHINGTON,NaN,DC,20007
3,I-200-16272-196340,WITHDRAWN,09,H-1B,17,20,INFO SERVICES LLC,NaN,17177 NORTH LAUREL PARK DR,LIVONIA,...,Year,Y,N,Y,NaN,NaN,JERSEY CITY,HUDSON,NJ,07302
4,I-200-15053-636744,CERTIFIED-WITHDRAWN,02,H-1B,15,18,BB&T CORPORATION,NaN,223 WEST NASH STREET,WILSON,...,Year,N,N,NaN,Y,NaN,NEW YORK,NEW YORK,NY,10036


In [133]:
raw_data.to_csv('h1b.csv', index=False)